In [ ]:
# Imports
%matplotlib widget
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import convolve2d

In [ ]:
# Read and show one image
im1 = cv2.imread("data/sample_input/butterfly.jpg", cv2.IMREAD_GRAYSCALE)
fig, axs = plt.subplots(1, 1, figsize=(7, 5))
axs.imshow(im1, cmap="gray")
axs.set_axis_off()
fig.suptitle("Original Image", fontsize=16)
plt.show()


In [ ]:
# Create X,Y Sobel filters and convolve the Image with them
SobX = np.array([[1, 0 , -1], [2, 0, -2], [1, 0, -1]])
SobY = SobX.T

fig, axs = plt.subplots(1, 2, figsize=(14, 5))
dX = convolve2d(im1, SobX, mode="full", fillvalue=0)
axs[0].imshow(dX, cmap="gray")
dY = convolve2d(im1, SobY, mode="full", fillvalue=0)
axs[1].imshow(dY, cmap="gray")
axs[0].set_axis_off()
axs[1].set_axis_off()
fig.suptitle("Sobel Convolution Outputs in both directions", fontsize=16)
plt.show()

In [ ]:
# Let's investigate what happens, when noise is added to the original image
im_noisy = im1 + np.random.randint(-80, 80, size=im1.shape)
fig, axs = plt.subplots(1, 1, figsize=(7, 5))
axs.imshow(im_noisy, cmap="gray")
axs.set_axis_off()
fig.suptitle("Original image with added noise", fontsize=16)
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(14, 5))
dX = convolve2d(im_noisy, SobX, mode="full", fillvalue=0)
axs[0].imshow(dX, cmap="gray")
dY = convolve2d(im_noisy, SobY, mode="full", fillvalue=0)
axs[1].imshow(dY, cmap="gray")
axs[0].set_axis_off()
axs[1].set_axis_off()
fig.suptitle("dX, dY with added noise", fontsize=16)
plt.show()

In [ ]:
# The derivative of a Gaussian is now constructed

def Sobel(dim):
    '''
    Return 3x3 Sobel filter of either X or Y direction depending on input dim
    '''
    if dim not in [0, 1]:
        raise Exception("dim must be either 0 or 1")
    if dim==0:
        return np.array([[-1, 0 , 1], [-2, 0, 2], [-1, 0, 1]])
    else:
        return np.array([[1, 2 , 1], [0, 0, 0], [-1, -2, -1]])
    

def gaussian_filter(sigma):
    '''
    Generate a gaussian filter of kernel size dynamically calculated based on sigma
    Returns: filter, kernel size
    '''
    dim = int(np.ceil(3 * sigma))
    if dim % 2 == 0:
        dim += 1
    X, Y = np.mgrid[-dim//2+1:dim//2+1, -dim//2+1:dim//2+1]
    gf = np.exp(-((X**2 + Y**2) / (2 * sigma**2))) / (2 * np.pi * sigma**2)
    
    return gf, dim


def DG(sigma):
    '''
    Generate the Derivative of a Gaussian based on the given Sigma
    Returns: Both X and Y derivative filters
    '''
    gf, dim = gaussian_filter(sigma)
    if dim <= 3:
        DGX = Sobel(dim=1)
        DGY = Sobel(dim=0)
    else:
        DGX = convolve2d(gf, Sobel(dim=1),mode="valid", boundary="symm")
        DGY = convolve2d(gf, Sobel(dim=0),mode="valid", boundary="symm")
    
    return DGX, DGY

def DG_filtering(im, sigma):
    '''
    Filter input sigma with Derivative of Gaussian filters on directions X and Y. Performs appropriate replicate padding to accomodate conservation of dimensions
    Returns: Filters, Derivatives, Gradient Magnitudes, Gradient Angles in (-pi/2, pi/2)
    '''
    DGX, DGY = DG(sigma=sigma)
    padding = int(DGX.shape[0] // 2)
    new_im = np.ndarray((im.shape[0] + 2 * padding, im.shape[1] + 2 * padding))
    new_im[padding:-padding, padding:-padding] = im
    new_im[padding:-padding, :padding] = np.repeat(im[:, 0].reshape(-1, 1), padding, axis=1)
    new_im[padding:-padding, -padding:] = np.repeat(im[:, 0].reshape(-1, 1), padding, axis=1)
    new_im[:padding, padding:-padding] = np.repeat(im[0, :].reshape(1, -1), padding, axis=0)
    new_im[-padding:, padding:-padding] = np.repeat(im[0, :].reshape(1, -1), padding, axis=0)
    new_im[:padding, :padding] = im[0, 0] * np.ones((padding, padding))
    new_im[:padding, -padding:] = im[0, -1] * np.ones((padding, padding))
    new_im[-padding:, :padding] = im[-1, 0] * np.ones((padding, padding))
    new_im[-padding:, -padding:] = im[-1, -1] * np.ones((padding, padding))

    dX = convolve2d(new_im, DGX, mode="valid")
    dY = convolve2d(new_im, DGY, mode="valid")
    magn = np.sqrt(dX**2 + dY**2)
    angle = np.arctan(dX / dY) 
    return dX, dY, DGX, DGY, magn, angle

In [ ]:
# Let's now plot the filters, the filtered Image the Gradient Magnitudes and Angles
dX_new_noiseless, dY_new_noiseless, DGX, DGY, magn_noiseless, angle_noiseless = DG_filtering(im1, sigma=3)

fig, axs = plt.subplots(1, 2, figsize=(14, 5))
axs[0].imshow(DGX)
axs[1].imshow(DGY)
axs[0].set_axis_off()
axs[1].set_axis_off()
axs[0].set_title("X Derivative of Gaussian Filter $\sigma=3$", fontsize=16)
axs[1].set_title("Y Derivative of Gaussian Filter $\sigma=3$", fontsize=16)
fig.savefig("Images/derivatives.svg", format="svg")
plt.show()


fig, axs = plt.subplots(1, 2, figsize=(14, 5))
axs[0].imshow(dX_new_noiseless, cmap="gray")
axs[1].imshow(dY_new_noiseless, cmap="gray")
axs[0].set_axis_off()
axs[1].set_axis_off()
axs[0].set_title("Noiseless X Derivative", fontsize=16)
axs[1].set_title("Noiseless Y Derivative", fontsize=16)
plt.show()

dX_new_noisy, dY_new_noisy, DGX, DGY, magn_noisy, angle_noisy = DG_filtering(im_noisy, sigma=3)

fig, axs = plt.subplots(1, 2, figsize=(14, 5))
axs[0].imshow(dX_new_noisy, cmap="gray")
axs[1].imshow(dY_new_noisy, cmap="gray")
axs[0].set_axis_off()
axs[1].set_axis_off()
axs[0].set_title("Noisy X Derivative", fontsize=16)
axs[1].set_title("Noisy Y Derivative", fontsize=16)
plt.show()

fig, axs = plt.subplots(1, 2, figsize=(14, 5))
axs[0].imshow(magn_noiseless, cmap="gray")
axs[1].imshow(magn_noisy, cmap="gray")
axs[0].set_axis_off()
axs[1].set_axis_off()
axs[0].set_title("Noiseless Gradient Magnitude", fontsize=16)
axs[1].set_title("Noisy Gradient Magnitude", fontsize=16)
plt.show()

fig, axs = plt.subplots(1, 2, figsize=(14, 5))
axs[0].imshow(angle_noiseless, cmap="gray")
axs[1].imshow(angle_noisy, cmap="gray")
axs[0].set_axis_off()
axs[1].set_axis_off()
axs[0].set_title("Noiseless Gradient Angles", fontsize=16)
axs[1].set_title("Noisy Gradient Angles", fontsize=16)
plt.show()

In [ ]:
# Now let us threshold the outputs
# First let's look at the histogram of the gradient magnitudes
fig, axs = plt.subplots(1, 2, figsize=(12, 7))
print(magn_noiseless.shape)
axs[0].hist(magn_noiseless.flatten(), bins=200)
axs[1].hist(magn_noisy.flatten(), bins=200)
axs[0].set_title("Noiseless Gradient Magnitude Histogram", fontsize=16)
axs[1].set_title("Noisy Gradient Magnitude Histogram", fontsize=16)
plt.show()

thresh = 38
thresh_noiseless = magn_noiseless.copy()
mask = thresh_noiseless <= thresh
thresh_noiseless[mask] = 0

thresh_noisy = magn_noisy.copy()
mask = thresh_noisy <= thresh
thresh_noisy[mask] = 0

# Plot the thresholded magnitude images
fig, axs = plt.subplots(1, 2, figsize=(14, 5))
axs[0].imshow(thresh_noiseless, cmap="gray")
axs[1].imshow(thresh_noisy, cmap="gray")
axs[0].set_axis_off()
axs[1].set_axis_off()
axs[0].set_title("Noiseless Thresholded Gradient Magnitude (thresh = {})".format(thresh), fontsize=14)
axs[1].set_title("Noisy Thresholded Gradient Magnitude (thresh = {})".format(thresh), fontsize=14)
plt.show()


In [ ]:
# Non-Maxima Suppresion
# Way 1: Based on 3x3 neighborhood regardless of angle

def non_maxima_suppresion_a(gradient_magnitudes):
    '''
    Perform Non-Maximaum-Suppression using Strategy A: (Suppress non maximum pixels regardless of the gradient angle)
    Returns: Suppressed filtered Image
    '''
    output = np.ndarray((gradient_magnitudes.shape[0] + 2, gradient_magnitudes.shape[1] + 2))
    output[1:-1, 1:-1] = gradient_magnitudes
    output[:, [0, -1]] = 0
    output[[0, -1], -1] = 0
    out_list = [0 if output[i, j] < np.max(output[i-1:i+2, j-1:j+2]) else output[i, j] for i in range(1, output.shape[0]-1) for j in range(1, output.shape[1] - 1)]
    out_array = np.array(out_list).reshape(gradient_magnitudes.shape[0], gradient_magnitudes.shape[1])
    output[1:-1, 1:-1] = out_array
    return output

# Way 2: Discretizing the angles in 4 bins, namely (0, 45, 90, 135) and performing non-maxima suppresion at these angles
def non_maxima_suppresion_b(gradient_magnitudes, gradient_angles):
    '''
    Perform Non-Maximaum-Suppression using Strategy B: (Suppress non maximum pixels in the direction of the quantized angle in bins of angle (0, 45, 90, 135))
    Returns: Suppressed filtered Image
    '''
    output = np.ndarray((gradient_magnitudes.shape[0], gradient_magnitudes.shape[1]))
    output[:, [0, -1]] = 0
    output[[0, -1], :] = 0
    output[1:-1, 1:-1] = gradient_magnitudes[1:-1, 1:-1]

    # Map angles
    for i in range(1, output.shape[0] - 1):
        for j in range(1, output.shape[1] - 1):
            ang = gradient_angles[i-1, j-1]
            if np.abs(ang) < np.pi / 8:
                if gradient_magnitudes[i, j] < np.max(gradient_magnitudes[i, j-1:j+2]):
                    output[i, j] = 0
            elif ang >= np.pi / 8 and ang < 3 * np.pi / 8:
                if gradient_magnitudes[i, j] < gradient_magnitudes[i-1, j+1] or gradient_magnitudes[i, j] < gradient_magnitudes[i+1, j-1]:
                    output[i, j] = 0
            elif ang <= - np.pi / 8 and ang > - 3 * np.pi / 8:
                if gradient_magnitudes[i, j] < gradient_magnitudes[i+1, j+1] or gradient_magnitudes[i, j] < gradient_magnitudes[i-1, j-1]:
                    output[i, j] = 0
            else:
                if gradient_magnitudes[i, j] < np.max(gradient_magnitudes[i-1:i+2, j]):
                    output[i, j] = 0
    
    return output


def non_maxima_suppresion_c(gradient_magnitudes, gradient_angles):
    '''
    Perform Non-Maximaum-Suppression using Strategy C: (Suppress non maximum pixels in the direction of the actual gradient angle using interpolated pixel intensities
    Returns: Suppressed filtered Image
    '''
    output = np.ndarray((gradient_magnitudes.shape[0], gradient_magnitudes.shape[1]))
    output[:, [0, -1]] = 0
    output[[0, -1], :] = 0
    output[1:-1, 1:-1] = gradient_magnitudes[1:-1, 1:-1]
    for i in range(1, output.shape[0] - 1):
        for j in range(1, output.shape[1] - 1):
            ang = gradient_angles[i-1, j-1]
            if ang > 0 and ang <= np.pi / 4:
                coef = np.tan(ang)
                p1 = (1 - coef) * gradient_magnitudes[i, j+1] + coef * gradient_magnitudes[i-1, j+1]
                p2 = (1 - coef) * gradient_magnitudes[i, j-1] + coef * gradient_magnitudes[i+1, j-1] 
                #print(coef, ang, "Case = 1")
                if gradient_magnitudes[i, j] < p1 or gradient_magnitudes[i, j] < p2:
                    output[i, j] = 0
            elif ang > np.pi / 4 and ang <= np.pi / 2:
                coef = np.tan(np.pi / 2 - ang)
                p1 = (1 - coef) * gradient_magnitudes[i-1, j] + coef * gradient_magnitudes[i-1, j+1]
                p2 = (1 - coef) * gradient_magnitudes[i+1, j] + coef * gradient_magnitudes[i+1, j-1] 
                #print(coef, ang, "Case = 2")
                if gradient_magnitudes[i, j] < p1 or gradient_magnitudes[i, j] < p2:
                    output[i, j] = 0
            elif ang <= 0 and ang > - np.pi / 4:
                coef = np.tan(np.abs(ang))
                p1 = (1 - coef) * gradient_magnitudes[i, j+1] + coef * gradient_magnitudes[i+1, j+1]
                p2 = (1 - coef) * gradient_magnitudes[i, j-1] + coef * gradient_magnitudes[i-1, j-1] 
                #print(coef, ang, "Case = 3")
                if gradient_magnitudes[i, j] < p1 or gradient_magnitudes[i, j] < p2:
                    output[i, j] = 0
            else:
                coef = np.tan(np.pi / 2 - np.abs(ang))
                p1 = (1 - coef) * gradient_magnitudes[i+1, j] + coef * gradient_magnitudes[i+1, j+1]
                p2 = (1 - coef) * gradient_magnitudes[i-1, j] + coef * gradient_magnitudes[i-1, j-1] 
                #print(coef, ang, "Case = 4")
                if gradient_magnitudes[i, j] < p1 or gradient_magnitudes[i, j] < p2:
                    output[i, j] = 0
            
           
    
    return output



out_a = non_maxima_suppresion_a(magn_noiseless)
out_b = non_maxima_suppresion_b(magn_noiseless, angle_noiseless)
out_c = non_maxima_suppresion_c(magn_noiseless, angle_noiseless)

fig, axs = plt.subplots(1, 1, figsize=(14, 8))
axs.imshow(out_a, cmap="gray")
axs.set_axis_off()
axs.set_title("Non-maximum-supression strategy-B", fontsize=16)
#fig.savefig("Images/edge_c.svg", format="svg")
plt.show()

fig, axs = plt.subplots(1, 1, figsize=(14, 8))
axs.imshow(out_b, cmap="gray")
axs.set_axis_off()
axs.set_title("Non-maximum-supression strategy-B", fontsize=16)
#fig.savefig("Images/edge_c.svg", format="svg")
plt.show()

fig, axs = plt.subplots(1, 1, figsize=(14, 8))
axs.imshow(out_c, cmap="gray")
axs.set_axis_off()
axs.set_title("Non-maximum-supression strategy-B", fontsize=16)
#fig.savefig("Images/edge_c.svg", format="svg")
plt.show()

In [ ]:
def connectivity_labeling(input_image):
    '''
    Adapt connectivity labeling algorithm for Canny Edge detection Hysterisis Thresholding
    Returns: Canny Edge Detection output
    '''
    label = 1
    flag_image = np.zeros_like(input_image)
    output_image = np.zeros_like(input_image)
    for i in range(flag_image.shape[0]):
        for j in range(flag_image.shape[1]):
            if input_image[i, j] == 2:
                if flag_image[i, j] == 0: # Pixel not yet labeled
                    # Label all component-connected pixels
                    flag_image[i, j] = label
                    queue = [(i, j)]
                    while queue:
                        tail = queue[-1]
                        queue.pop()
                        new_queue = [(k, n) for k in range(tail[0]-1, tail[0]+2) for n in range(tail[1]-1, tail[1]+2) if (k != tail[0] or n != tail[1])
                                     and k >= 0 and k <= flag_image.shape[0] - 1
                                      and n >= 0 and n <= flag_image.shape[1] - 1 and (input_image[k, n] == 1 or input_image[k, n] == 2) and flag_image[k, n] == 0]
                        for pix in new_queue: 
                            flag_image[pix] = label
                            output_image[pix] = 2
                        queue = new_queue + queue

    return output_image



def Canny(img, thresh1, thresh2, sigma):
    '''
    Canny Edge Detection from scratch
    Inputs: threshold1, threshold2 (t1 < t2), sigma for Bluring
    Returns: Canny Edge Detection Output
    '''
    dX, dY, DGX, DGY, magn, angle = DG_filtering(img, sigma=sigma)
    thresh1 = np.std(img) / 255 * thresh1
    thresh2 = np.std(img) / 255 * thresh2
    out = non_maxima_suppresion_c(magn, angle)
    out_thresh = np.zeros_like(out)
    out_thresh[np.logical_and(out >= thresh1, out <= thresh2)] = 1
    out_thresh[out > thresh2] = 2
    out_final = connectivity_labeling(out_thresh)
    return out_final, out_thresh


In [ ]:
# Get some Images and display the Canny Edge Detection Output
images = []
im = cv2.imread("data/sample_input/okeefe.jpg", cv2.IMREAD_GRAYSCALE)
im = cv2.resize(im, (int(im.shape[1] * 0.3), int(im.shape[0] * 0.3)))
images.append(im)
im = cv2.imread("data/sample_input/kusama.jpg", cv2.IMREAD_GRAYSCALE)
im = cv2.resize(im, (int(im.shape[1] * 0.85), int(im.shape[0] * 0.85)))
images.append(im)
im = cv2.imread("data/sample_input/khalo.jpg", cv2.IMREAD_GRAYSCALE)
im = cv2.resize(im, (int(im.shape[1] * 0.7), int(im.shape[0] * 0.7)))
images.append(im)
im = cv2.imread("data/sample_input/lasnig.png", cv2.IMREAD_GRAYSCALE)
images.append(im)


fig, axs = plt.subplots(4, 2, figsize=(12, 18))
for i, image in enumerate(images):
    out_final, out_thresh = Canny(image, 50, 120, 3.5)
    axs[i, 0].imshow(out_thresh, cmap="gray")
    axs[i, 1].imshow(out_final, cmap="gray")
    axs[i, 0].set_axis_off()
    axs[i, 1].set_axis_off()
fig.suptitle("Canny Edge Detection for all given images and $(t_1 = 50, t_2=120)$")
fig.savefig("Images/paintings_50120.svg", format="svg")
plt.show()


In [ ]:
# Blob detection
def generate_log_kernel(sigma, norm=True):
    '''
    Generates the Laplacian of a Gaussian Kernel based on the given sigma value. The kernel size is dynamically defined. If norm is set to True the Kernel is normalized 
    Returns: LoG Filter
    '''
    # Ensure size is odd
    size = int(np.ceil(4 * sigma))
    if size % 2 == 0:
        size += 1
    x, y = np.mgrid[-size:size+1, -size:size+1]
    g = -1/ (np.pi * sigma**4) * (1 - (x**2 + y**2) / (2 * sigma**2)) * np.exp(-(x**2 + y**2) / (2 * sigma**2))
    if norm:
        g *= sigma**2
    return g

def replicate_padding(im, filter):
    '''
    Function that performs replicate padding given the input image and filter
    Returns: Padded Image
    '''
    padding = int(filter.shape[0] // 2)
    new_im = np.ndarray((im.shape[0] + 2 * padding, im.shape[1] + 2 * padding))
    new_im[padding:-padding, padding:-padding] = im
    new_im[padding:-padding, :padding] = np.repeat(im[:, 0].reshape(-1, 1), padding, axis=1)
    new_im[padding:-padding, -padding:] = np.repeat(im[:, -1].reshape(-1, 1), padding, axis=1)
    new_im[:padding, padding:-padding] = np.repeat(im[0, :].reshape(1, -1), padding, axis=0)
    new_im[-padding:, padding:-padding] = np.repeat(im[-1, :].reshape(1, -1), padding, axis=0)
    new_im[:padding, :padding] = im[0, 0] * np.ones((padding, padding))
    new_im[:padding, -padding:] = im[0, -1] * np.ones((padding, padding))
    new_im[-padding:, :padding] = im[-1, 0] * np.ones((padding, padding))
    new_im[-padding:, -padding:] = im[-1, -1] * np.ones((padding, padding))

    return new_im

def LoG_filter(im, sigma0, scale_factor, n):
    '''
    Perform LoG Filtering, of the input image using a set of filters defined by the initial sigma0 and sigmas generated by recursively scaling sigma0 n-times
    Returns: Filtered Images, Blob Radius
    '''
    sigmas = [sigma0 * (np.power(scale_factor, i)) for i in range(n)]
    radius = [np.sqrt(2) * sigma for sigma in sigmas]
    filters = [generate_log_kernel(sigma) for sigma in sigmas]
    filtered_images = []
    for i, filter in enumerate(filters):
        new_im = replicate_padding(im, filter)
        filt_im = np.power(convolve2d(new_im, filter, mode="valid"), 2)
        filt_im /= np.std(filt_im)
        filtered_images.append(filt_im)

    return filtered_images, radius

def harris_response_for_neighborhood(neighborhood, k=0.04):
    '''
    Perform the Harris Transform for Corner-Edge Detection, for the center of the given neighborhood and the k-constant
    Returns: Harris Value for center pixel
    '''
    # Compute gradients (Sobel operators applied to the neighborhood)
    # Note: Normally, we would use cv2.Sobel or similar on a larger image. Here, we approximate this for the 3x3 case.
    dx = Sobel(dim=0)
    dy = Sobel(dim=1)
    Ix = np.sum(neighborhood * dx)
    Iy = np.sum(neighborhood * dy)
    
    # Compute products of derivatives at every pixel
    Ixx = Ix**2
    Ixy = Ix*Iy
    Iyy = Iy**2
    
    # Sum of products of derivatives for the pixels in the window
    # For a 3x3 neighborhood, it's just the value itself, no sum needed.
    
    # Compute the determinant and trace of the matrix M
    detM = Ixx * Iyy - Ixy**2
    traceM = Ixx + Iyy
    
    # Calculate Harris response
    R = detM - k * traceM**2
    
    return R


def harris_response_for_image(image, k):
    '''
    Perform the Harris Transform for Corner-Edge Detection, for the entire Image given the constant k
    Returns: Harris Values for the entire Image
    '''
    # Convert image to grayscale if it's not already
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image

    gray_filtered = cv2.GaussianBlur(gray, (5, 5), 0)

    # Compute gradients
    Ix = cv2.Sobel(gray_filtered, cv2.CV_64F, 1, 0, ksize=5)
    Iy = cv2.Sobel(gray_filtered, cv2.CV_64F, 0, 1, ksize=5)

    # Compute products of derivatives
    Ixx = Ix**2
    Ixy = Ix*Iy
    Iyy = Iy**2

    # Apply Gaussian window (w)
    gIxx = cv2.GaussianBlur(Ixx, (3, 3), 0)
    gIxy = cv2.GaussianBlur(Ixy, (3, 3), 0)
    gIyy = cv2.GaussianBlur(Iyy, (3, 3), 0)

    # Compute determinant and trace of the matrix M
    detM = gIxx * gIyy - gIxy**2
    traceM = gIxx + gIyy
    
    # Compute Harris response
    R = detM - k * traceM**2

    # Determine if (x, y) is a corner
    return R


def scale_space_non_maxima_suppression(image, filtered_scale_space, radius, thresh, harris_thresh):
    '''
    Performs Non-Maximum-Suppression for the given scale-space 3D Tensor by sliding a 3x3x3 cube around and asserting whether the center pixel is the maximum or not for the given neighborhood
    Returns: Detected Blob coordinates, Blobs associated to flat regions given Harris value, Blobs associated with edge points given Harris value
    '''
    scale_space = np.array(filtered_scale_space)
    blobs = np.ones_like(filtered_scale_space)
    blob_points = []
    harris_edges = []
    harris_flats = []
    harris_response = harris_response_for_image(image, k=0.04)

    for i in range(1, len(filtered_scale_space)-1):
        for x in range(1, blobs.shape[1]-1):
            for y in range(1, blobs.shape[2]-1):
                if scale_space[i, x, y] < np.max(scale_space[i-1:i+2, x-1:x+2, y-1:y+2]):
                    blobs[i, x, y] = 0
                else:
                    if scale_space[i, x, y] > thresh:
                        harris = harris_response[x, y]
                        if harris > harris_thresh:
                            blob_points.append((x, y, radius[i], i))
                            harris_flats.append((x, y))
                        else:
                            harris_edges.append((x, y))

    return blob_points, harris_flats, harris_edges

def LoG_filter_b(im, sigma0, scale_factor, n):
    '''
    Performs LoG Filtering using strategy B, by iteratively downscaling the Image instead of Upscaling the LoG filter
    Returns: Scale-Space filters, BloB radius
    '''
    sigmas = [sigma0 * np.power((1 / scale_factor), i) for i in range(n)]
    radius = [3 * sigma for sigma in sigmas]
    dim0 = (im.shape[1], im.shape[0])
    filter = generate_log_kernel(sigma0, norm=False)
    image_scale_space = []
    filtered_scale_space = []
    temp = im.copy()
    for i in range(n):
        image_scale_space.append(temp)
        temp = cv2.GaussianBlur(temp,(3,3), 0)
        dim = (int(scale_factor * temp.shape[1]), int(scale_factor * temp.shape[0]))
        temp = cv2.resize(temp, (dim), interpolation = cv2.INTER_LINEAR)
        filtered_temp = convolve2d(replicate_padding(temp, filter), filter, mode="valid")
        filtered_temp = np.power(filtered_temp, 2) 
        filtered_temp /= np.std(filtered_temp)
        filtered_scale_space.append(cv2.resize(filtered_temp, dim0, interpolation = cv2.INTER_LANCZOS4 ))
    

    return filtered_scale_space, radius

In [ ]:
# Generate some LoG kernels and visualize them
sigmas = [2, 4, 8, 12]
fig, axs = plt.subplots(2, 2, figsize=(10, 10))
for i, sig in enumerate(sigmas):
    axs[i%2, i//2].imshow(generate_log_kernel(sig, norm=False))
    axs[i%2, i//2].set_axis_off()
    axs[i%2, i//2].set_title("LoG filter $\sigma={}$".format(sig), fontsize=16)

#fig.savefig("Images/log_filters.svg", format="svg")
plt.show()

In [ ]:
im = cv2.imread("data/sample_input/einstein.jpg", cv2.IMREAD_GRAYSCALE)
sigma0 = 1.2
scale_factor = 1.3
n = 12
filtered_images, radius = LoG_filter(im, sigma0, scale_factor, n)
harris_responses = [harris_response_for_image(filt_image, k=0.04) for filt_image in filtered_images]
fig, axs = plt.subplots(4, 3, figsize=(18, 25))
for i in range(n):
    axs[i//3, i%3].imshow(filtered_images[i], cmap="jet")
    axs[i//3, i%3].set_axis_off()

#fig.savefig("Images/einstein.svg", format="svg")
plt.show()

In [ ]:
im = cv2.imread("data/sample_input/einstein.jpg", cv2.IMREAD_GRAYSCALE)
sigma0 = 1.8
scale_factor = 1.25
n = 15
filtered_images, radius = LoG_filter(im, sigma0, scale_factor, n)
blob_points, harris_flats, harris_edges = scale_space_non_maxima_suppression(im, filtered_images, radius, 1, harris_thresh=-1e9)


In [ ]:
red = [255,0,0]
blue = [0, 0, 255]
green = [0, 255, 0]

rgb_image = cv2.cvtColor(im.copy(), cv2.COLOR_GRAY2BGR)
rgb_image_copy = rgb_image.copy()
for (x, y) in harris_edges:
    cv2.circle(rgb_image_copy, (y, x), 4, blue, -1)

fig, axs = plt.subplots(1, 1, figsize=(10, 6))
axs.imshow(rgb_image_copy)
axs.set_axis_off()
axs.set_title("Image and Harris Rejected blob points", fontsize=16)
fig.savefig("Images/harris_rejected_einstein.svg", format="svg")
plt.show()



fig, axs = plt.subplots(5, 3, figsize=(18, 27))


for j, filt_im in enumerate(filtered_images):
    filt_normalized = cv2.normalize(filt_im, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    filt_uint8 = np.uint8(filt_normalized)
    filt_rgb = cv2.cvtColor(filt_uint8, cv2.COLOR_GRAY2BGR)
    for (x, y, r, i) in blob_points:
        if i == j:
            cv2.circle(filt_rgb, (y, x), 4, red, -1)
    axs[j //3, j%3].imshow(filt_rgb)

    axs[j //3, j%3].set_axis_off()

fig.suptitle("Blob points in their corresponding filtered Images", fontsize=16)
fig.savefig("Images/blobs_filters_einstein.svg", format="svg")
plt.show()

for (x, y, r, i) in blob_points:
    cv2.circle(rgb_image, (y, x), int(np.ceil(r)), red, 1)

fig, axs = plt.subplots(1, 1, figsize=(10, 6))
axs.imshow(rgb_image)
axs.set_axis_off()
axs.set_title("Detected Blobs", fontsize=16)
fig.savefig("Images/blobs_einstein.svg", format="svg")
plt.show()



resp = harris_response_for_image(im, k=0.04)
'''
resp_normalized = cv2.normalize(resp, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
resp_uint8 = np.uint8(resp_normalized)
resp_rgb = cv2.cvtColor(resp_uint8, cv2.COLOR_GRAY2BGR)
'''

fig, axs = plt.subplots(1, 1, figsize=(10, 6))
axs.imshow(resp)
axs.set_axis_off()
axs.set_title("Harris output on initial Image", fontsize=16)
fig.savefig("Images/harris_output_einstein.svg", format="svg")
plt.show()

In [ ]:
im = cv2.imread("data/sample_input/khalo.jpg", cv2.IMREAD_GRAYSCALE)
im = cv2.resize(im, (int(im.shape[1] * 0.5), int(im.shape[0] * 0.5)))
im_rgb = cv2.imread("data/sample_input/khalo.jpg")
im_rgb = cv2.resize(im_rgb, (int(im_rgb.shape[1] * 0.5), int(im_rgb.shape[0] * 0.5)))
im_rgb = cv2.cvtColor(im_rgb, cv2.COLOR_BGR2RGB)
sigma0 = 0.8
scale_factor = 0.85
n = 23
filtered_images, radius = LoG_filter_b(im, sigma0, scale_factor, n)    
blob_points, harris_flats, harris_edges = scale_space_non_maxima_suppression(im, filtered_images, radius, thresh=0.4, harris_thresh=-1e8)
blue = [0,0,255]
red = [255, 0, 0]
#rgb_image = cv2.cvtColor(im.copy(), cv2.COLOR_GRAY2BGR)
for (x, y, r, i) in blob_points:
    cv2.circle(im_rgb, (y, x), int(np.ceil(r)), blue, 1)
fig, axs = plt.subplots(1, 1, figsize=(10, 6))
axs.imshow(im_rgb)
axs.set_axis_off()
plt.show()
fig.savefig("Images/blobs_khalo_b.svg", format="svg")
